In [1]:
import pandas as pd
import requests
import time
import calendar
import sys

# 1. 파일 이름 정의
INPUT_FILE = 'combined_sorted_earthquakes.csv'
OUTPUT_FILE = 'combined_with_match_count.csv'
USGS_API_URL = "https://earthquake.usgs.gov/fdsnws/event/1/query"

def get_last_day_of_month(year, month):
    """지정한 연도와 월의 마지막 날짜를 반환합니다."""
    return calendar.monthrange(year, month)[1]

def get_usgs_match_count(row):
    """
    API에 쿼리하여 매칭되는 지진의 '개수'를 반환합니다.
    """
    # 1. API 파라미터 계산
    year = int(row['Year'])
    month = int(row['Month'])
    last_day = get_last_day_of_month(year, month)

    # 2. 날짜 형식 설정 (YYYY-MM-DD)
    start_time = f"{year}-{month:02d}-01"
    end_time = f"{year}-{month:02d}-{last_day:02d}"

    # 3. 규모 범위 설정
    min_magnitude = row['magnitude'] - 0.4
    max_magnitude = row['magnitude'] + 0.4

    params = {
        'format': 'geojson',
        'starttime': start_time,
        'endtime': end_time,
        'latitude': row['latitude'],
        'longitude': row['longitude'],
        'maxradiuskm': 10,  # 요청하신 대로 10km 반경
        'minmagnitude': min_magnitude,
        'maxmagnitude': max_magnitude
    }

    # 4. API 호출 (차단을 피하기 위해 1초 대기)
    #    (이것이 스크립트가 오래 걸리는 이유입니다)
    time.sleep(1)

    try:
        # API 요청 (15초 타임아웃)
        response = requests.get(USGS_API_URL, params=params, timeout=15)
        # 4xx, 5xx 에러가 발생하면 예외 발생
        response.raise_for_status()

        data = response.json()

        # GeoJSON의 'metadata.count' 필드가 바로 지진 개수임
        count = data['metadata']['count']

        print(f"  > [행 {row.name}] 성공: {count}개 발견 (Y:{year}, M:{month}, Mag:{row['magnitude']:.1f})")
        return count

    except requests.exceptions.HTTPError as e:
        print(f"  > [행 {row.name}] API HTTP 오류: {e.response.status_code}")
        return -1 # 에러 코드 -1
    except requests.exceptions.RequestException as e:
        print(f"  > [행 {row.name}] API 요청 오류: {e}")
        return -1 # 에러 코드 -1
    except Exception as e:
        print(f"  > [행 {row.name}] 알 수 없는 오류: {e}")
        return -1 # 에러 코드 -1

def main():
    try:
        df = pd.read_csv(INPUT_FILE)
    except FileNotFoundError:
        print(f"[오류] '{INPUT_FILE}'을(를) 찾을 수 없습니다.", file=sys.stderr)
        return
    except Exception as e:
        print(f"[오류] 파일을 읽는 중 오류 발생: {e}", file=sys.stderr)
        return

    total_rows = len(df)
    print(f"총 {total_rows}개의 지진 데이터에 대한 USGS API 조회를 시작합니다.")
    print(f"각 호출당 1초씩 대기하므로, 예상 소요 시간은 약 {total_rows // 60}분 입니다...")
    print("="*50)

    # 1. 'usgs_match_count' 컬럼을 생성합니다.
    #    .apply(axis=1)을 사용하여 각 행(row)에 대해 get_usgs_match_count 함수를 실행합니다.
    df['usgs_match_count'] = df.apply(get_usgs_match_count, axis=1)

    print("="*50)
    print("API 조회가 완료되었습니다.")

    # 2. 결과 요약
    print("\n--- 'usgs_match_count' 컬럼 결과 요약 ---")
    print(df['usgs_match_count'].value_counts(dropna=False).sort_index().to_markdown())

    # 3. 새 CSV 파일로 저장
    try:
        df.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')
        print(f"\n[성공] 'usgs_match_count' 컬럼이 추가된 파일이")
        print(f"'{OUTPUT_FILE}'(으)로 저장되었습니다.")
    except PermissionError:
        print(f"\n[오류] '{OUTPUT_FILE}'을(를) 저장할 수 없습니다.")
        print("     파일이 다른 프로그램에서 열려있는지 확인하세요.")

if __name__ == "__main__":
    main()

총 1099개의 지진 데이터에 대한 USGS API 조회를 시작합니다.
각 호출당 1초씩 대기하므로, 예상 소요 시간은 약 18분 입니다...
  > [행 0] 성공: 10개 발견 (Y:2024, M:1, Mag:5.1)
  > [행 1] 성공: 8개 발견 (Y:2024, M:1, Mag:5.5)
  > [행 2] 성공: 1개 발견 (Y:2024, M:1, Mag:5.4)
  > [행 3] 성공: 1개 발견 (Y:2024, M:9, Mag:5.1)
  > [행 4] 성공: 1개 발견 (Y:2024, M:8, Mag:6.1)
  > [행 5] 성공: 0개 발견 (Y:2024, M:8, Mag:5.4)
  > [행 6] 성공: 2개 발견 (Y:2024, M:7, Mag:5.1)
  > [행 7] 성공: 0개 발견 (Y:2024, M:7, Mag:6.2)
  > [행 8] 성공: 0개 발견 (Y:2024, M:5, Mag:6.4)
  > [행 9] 성공: 1개 발견 (Y:2024, M:4, Mag:5.4)
  > [행 10] 성공: 1개 발견 (Y:2024, M:4, Mag:5.6)
  > [행 11] 성공: 1개 발견 (Y:2024, M:4, Mag:5.5)
  > [행 12] 성공: 1개 발견 (Y:2024, M:3, Mag:5.6)
  > [행 13] 성공: 1개 발견 (Y:2024, M:3, Mag:5.6)
  > [행 14] 성공: 0개 발견 (Y:2024, M:8, Mag:5.1)
  > [행 15] 성공: 1개 발견 (Y:2024, M:8, Mag:5.1)
  > [행 16] 성공: 0개 발견 (Y:2024, M:8, Mag:5.0)
  > [행 17] 성공: 1개 발견 (Y:2024, M:8, Mag:5.6)
  > [행 18] 성공: 1개 발견 (Y:2024, M:4, Mag:5.6)
  > [행 19] 성공: 1개 발견 (Y:2024, M:3, Mag:5.8)
  > [행 20] 성공: 1개 발견 (Y:2024, M:3, Mag:5.4)
  > [